In [1]:
%load_ext autoreload
%autoreload 2

import sys
import os
import time

sys.path.append(os.environ['REPO_DIR'] + '/utilities')
from utilities2015 import *
from metadata import *
from data_manager import *
from learning_utilities import *

Setting environment for Precision WorkStation


No vtk
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Based on human annotation

In [2]:
windowing_settings

{1: {'patch_size': 224, 'spacing': 56},
 2: {'comment': 'larger margin', 'patch_size': 224, 'spacing': 56},
 3: {'comment': 'smaller spacing', 'patch_size': 224, 'spacing': 32},
 4: {'comment': 'smaller spacing', 'patch_size': 224, 'spacing': 128},
 5: {'comment': 'smaller spacing', 'patch_size': 224, 'spacing': 64},
 6: {'comment': 'twice as large patch', 'patch_size': 448, 'spacing': 64},
 7: {'comment': 'specify size/spacing in terms of microns rather than pixels',
  'patch_size_um': 103.04,
  'spacing_um': 30},
 8: {'comment': 'larger patch', 'patch_size_um': 206.08, 'spacing_um': 30},
 9: {'comment': 'larger patch', 'patch_size_um': 412.16, 'spacing_um': 30},
 10: {'comment': 'larger patch', 'patch_size_um': 824.32, 'spacing_um': 30}}

In [ ]:
win_id = 7
for stack in ['MD585', 'MD594', 'MD589']:
    grid_index_class_lookup, latest_timestamp = generate_annotation_to_grid_indices_lookup(stack, win_id=win_id, by_human=True, 
                                                                         suffix='contours', timestamp='latest',
                                                                         surround_margins=[200, 500],
                                                                        return_timestamp=True)
    
    grid_index_class_lookup_fp = \
    DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, 
                                                               by_human=True, timestamp=latest_timestamp)
    print grid_index_class_lookup_fp

    save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
    upload_to_s3(grid_index_class_lookup_fp)

In [ ]:
# win_id = 3
# windowing_properties = windowing_settings[win_id]
# patch_size = windowing_properties['patch_size']
# spacing = windowing_properties['spacing']
# w, h = metadata_cache['image_shape'][stack]
# half_size = patch_size/2
# grid_spec = (patch_size, spacing, w, h)

# TEST

In [ ]:
from shapely.geometry import Polygon, Point

In [ ]:
warped_volumes = DataManager.load_transformed_volume_all_known_structures(stack_m='atlasV5', 
                                                                      stack_f='MD662',
                                                                        detector_id_f=1,
                                                                      prep_id_f=2,
                                                                        warp_setting=17, 
                                                                      sided=True)

In [ ]:
area_all_levels = defaultdict(lambda: defaultdict(dict))
positive_patch_numbers_all_levels = defaultdict(lambda: defaultdict(dict))

stack = 'MD662'
win_id = 5
surround_margins = [200]

for level in np.arange(0, 1, 0.1):
    structure_contours_pos_level = \
    get_structure_contours_from_aligned_atlas({'3N_L': warped_volumes['3N_L']}, volume_origin=(0,0,0), 
                                              sections=metadata_cache['valid_sections'][stack], 
                                              downsample_factor=32, level=level, 
                                              sample_every=1, 
                                              first_sec=metadata_cache['section_limits'][stack][0])

    for sec, x in structure_contours_pos_level.iteritems():
        for name_s, v in x.iteritems():
            area_all_levels[name_s][sec][level] = Polygon(v).area
            
    grid_spec = win_id_to_gridspec(win_id, stack=stack)

    sections = structure_contours_pos_level.keys()

    for sec in sections:
        sys.stderr.write('Computing grid indices lookup for section %d...\n' % sec)
        if is_invalid(sec=sec, stack=stack):
            continue

        mask_tb = DataManager.load_thumbnail_mask_v3(stack=stack, section=sec, prep_id=2)

        q = locate_patches_v2(grid_spec=grid_spec, mask_tb=mask_tb, \
                              polygons=structure_contours_pos_level[sec].items(), \
                          surround_margins=surround_margins, categories=['positive'])

        for name_s, grid_ids in q.iteritems():
            positive_patch_numbers_all_levels[name_s][sec][level] = len(grid_ids)
            
save_json(positive_patch_numbers_all_levels, '/tmp/positive_patch_numbers_all_levels.json')
save_json(area_all_levels, '/tmp/area_all_levels.json')

In [ ]:
# positive_patch_numbers_all_levels = load_json('/tmp/positive_patch_numbers_all_levels.json')

In [ ]:
for name_s, x in positive_patch_numbers_all_levels.iteritems():
    for sec, y in sorted(x.items()):
        y2 = {float('%.2f' % float(level)): yy for level, yy in sorted(y.items())}
        print sec, sorted(y.keys())[-1], sorted(y2.items())

In [ ]:
for level, x in area_all_levels.iteritems():
    for sec, y in x.iteritems():
        for name_s, a in y.iteritems():
            if name_s == '10N_L':
                print name_s, sec, a

# TEST END

# Based on aligned atlas

In [ ]:
positive_level = {}
for name_u in all_known_structures:
    if name_u == '4N':
        positive_level[name_u] = 0.6
    elif name_u == '10N':
        positive_level[name_u] = 0.7
    else:
        positive_level[name_u] = 0.8
        
# positive_level = 0.8

In [ ]:
from learning_utilities import *

In [ ]:
# stack = 'ChatCryoJane201710'
for stack in [
 'MD652',
 'MD653',
 'MD657',
'MD658']:
    win_id = 5
    detector_id = 13
    grid_index_class_lookup = generate_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id, by_human=False, 
                                                                                    detector_id_f=detector_id,
                                                                        surround_margins=[200],
                                                                        positive_level=0.8, negative_level=0.1)
    grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=False, 
                                                                     stack_m='atlasV5', 
                                                                                detector_id_f=detector_id,
                                                          timestamp='now')
    save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
    upload_to_s3(grid_index_class_lookup_fp)

In [ ]:
# stack = 'ChatCryoJane201710'
for stack in [
 'MD661',
 'MD662']:
    win_id = 7
    detector_id = 1
    grid_index_class_lookup = generate_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id, by_human=False, 
                                                                                    detector_id_f=detector_id,
                                                                        surround_margins=[200],
                                                                        positive_level=positive_level, 
                                                                         negative_level=0.1)
    grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=False, 
                                                                     stack_m='atlasV5', 
                                                                                detector_id_f=detector_id,
                                                          timestamp='now')
    save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
    upload_to_s3(grid_index_class_lookup_fp)

# Use human-corrected structure annotation file

In [ ]:
positive_level = {}
for name_u in all_known_structures:
    if name_u == '4N':
        positive_level[name_u] = 0.6
    elif name_u == '10N':
        positive_level[name_u] = 0.7
    else:
        positive_level[name_u] = 0.8
        
# positive_level = 0.8

In [ ]:
# stack = 'ChatCryoJane201710'
for stack in [
 'MD661']:
    win_id = 7
    grid_index_class_lookup = generate_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id, by_human=True, 
                                                                        surround_margins=[200, 500],
                                                                        positive_level=positive_level, 
                                                                         negative_level=0.1,
                                                                        suffix='structures')
    grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=True, 
                                                           timestamp='02062018184326',
                                                          suffix='structures')
    
    save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
    upload_to_s3(grid_index_class_lookup_fp)

In [ ]:
grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=True, 
                                                       timestamp='02062018184326',
                                                      suffix='structures')

save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
upload_to_s3(grid_index_class_lookup_fp)

In [ ]:
%load_ext autoreload
%autoreload 2

from data_manager import *

In [ ]:
grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=True, 
                                                           timestamp='02062018184326',
                                                          suffix='structures')

In [ ]:
grid_index_class_lookup_fp

# Examine

In [ ]:
%matplotlib inline

In [ ]:
sec = 150

In [ ]:
# img = DataManager.load_image_v2(stack=stack, section=sec, prep_id=2, version='gray')
img = DataManager.load_image_v2(stack=stack, section=sec, prep_id=2)

In [ ]:
viz = img.copy()

# for s in ['VCA', 'VCP']:
for s in ['7N', '7N_surround_200_noclass', 
          '5N_surround_200_noclass',
         'Pn_surround_200_noclass']:
    all_locs = grid_parameters_to_sample_locations(win_id=win_id, stack=stack)
    if not sec in grid_index_class_lookup[s].dropna():
        sys.stderr.write("No %s patches on section %d.\n" % (s, sec))
        continue
    for x, y in all_locs[grid_index_class_lookup[s][sec]]:
        cv2.circle(viz, (x,y), 20, (255,0,0), 3)

In [ ]:
# viz = img.copy()

# # for s in ['VCA', 'VCP']:
# for s in ['7N_L', '7N_L_surround_200_noclass', 
#           '5N_L_surround_200_noclass',
#          'Pn_L_surround_200_noclass']:
#     all_locs = grid_parameters_to_sample_locations(win_id=win_id, stack=stack)
#     if not sec in grid_index_class_lookup[s].dropna():
#         sys.stderr.write("No %s patches on section %d.\n" % (s, sec))
#         continue
#     for x, y in all_locs[grid_index_class_lookup[s][sec]]:
#         cv2.circle(viz, (x,y), 20, (255,0,0), 3)

In [ ]:
display_image(viz[::4, ::4])

# Save and Upload

In [ ]:
grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, 
                                                           by_human=True, timestamp=latest_timestamp)
print grid_index_class_lookup_fp

In [ ]:
grid_index_class_lookup_fp = \
DataManager.get_annotation_to_grid_indices_lookup_filepath(stack=stack, win_id=win_id, by_human=False, 
                                                                     stack_m='atlasV5', 
                                                                                detector_id_f=detector_id,
                                                          timestamp='now')

In [ ]:
save_hdf_v2(grid_index_class_lookup, grid_index_class_lookup_fp)
upload_to_s3(grid_index_class_lookup_fp)

In [ ]:
stack = 'MD595'

In [ ]:
grid_index_class_lookup = \
DataManager.load_annotation_to_grid_indices_lookup(stack=stack, win_id=win_id, by_human=False, 
                                                                     stack_m=atlas_name, 
                                                                                detector_id_f=detector_id,
                                                           prep_id_f=2,
                                                                                warp_setting=warp_setting,
                                                          timestamp='latest')

In [ ]:
grid_index_class_lookup = DataManager.load_annotation_to_grid_indices_lookup(stack='MD589', win_id=5, 
                                                                             by_human=True,
                                                                            timestamp='latest')

In [ ]:
grid_index_class_lookup.loc[108]['VCA']

In [ ]:
# for stack in all_annotated_stacks:
    
#     sys.stderr.write('%s\n' % stack)
    
#     label_gridIndices = locate_annotated_patches_v2(stack=stack)
    
#     annotation_grid_indices_fn = os.path.join(ANNOTATION_ROOTDIR, stack, '%(stack)s_annotation_grid_indices.h5' % dict(stack=stack))
#     label_gridIndices.to_hdf(annotation_grid_indices_fn, 'grid_indices')

In [ ]:
# label_gridIndices = read_hdf(annotation_grid_indices_fn, 'grid_indices')